In [1]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [2]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit, substring, avg, sum, round

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/gold/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fd473c43-58b3-4a33-ae9f-c18de688465c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 234ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      d

In [3]:
# apagar tudo do hdfs

to_path = "/projeto/fisico/gold/speed_distance/"
#to_path_silver = "/projeto/fisico/gold/speed_distance/"

client.delete(to_path,True)
#client.delete(to_path_silver,True)

False

In [4]:
# read data from the silver tables
df = spark.table("silver.speed_distance")

In [5]:
#df.show(truncate=False)
df.printSchema()
df.toPandas()

root
 |-- PLAYER: string (nullable = true)
 |-- TEAM: string (nullable = true)
 |-- SEASON: string (nullable = true)
 |-- GP: integer (nullable = true)
 |-- W: integer (nullable = true)
 |-- L: integer (nullable = true)
 |-- MIN: integer (nullable = true)
 |-- AVG_MIN_PER_GAME: double (nullable = true)
 |-- DIST_FEET: integer (nullable = true)
 |-- DIST_MILES: integer (nullable = true)
 |-- DIST_MILES_OFF: double (nullable = true)
 |-- DIST_MILES_DEF: double (nullable = true)
 |-- DIST_KM: double (nullable = true)
 |-- DIST_KM_OFF: double (nullable = true)
 |-- DIST_KM_DEF: double (nullable = true)
 |-- DIST_KM_PER_MIN: double (nullable = true)
 |-- DIST_KM_OFF_PER_MIN: double (nullable = true)
 |-- DIST_KM_DEF_PER_MIN: double (nullable = true)
 |-- AVG_SPEED_MPH: double (nullable = true)
 |-- AVG_SPEED_MPH_OFF: double (nullable = true)
 |-- AVG_SPEED_MPH_DEF: double (nullable = true)
 |-- AVG_SPEED_KMH: double (nullable = true)
 |-- AVG_SPEED_KMH_OFF: double (nullable = true)
 |-- AVG

22/01/15 15:04:37 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


PLAYER TEAM     SEASON  GP   W   L   MIN  AVG_MIN_PER_GAME  \
0       Aaron Brooks  MIN  2017/2018  32  19  13   189          5.906250   
1       Aaron Gordon  ORL  2017/2018  58  19  39  1909         32.913793   
2     Aaron Harrison  DAL  2017/2018   9   2   7   233         25.888889   
3      Aaron Jackson  HOU  2017/2018   1   0   1    35         35.000000   
4        Abdel Nader  BOS  2017/2018  47  32  15   520         11.063830   
...              ...  ...        ...  ..  ..  ..   ...               ...   
4026     Willie Reed  BKN  2015/2016  39   8  31   426         10.923077   
4027  Xavier Munford  MEM  2015/2016  14   3  11   244         17.428571   
4028     Zach LaVine  MIN  2015/2016  82  29  53  2293         27.963415   
4029   Zach Randolph  MEM  2015/2016  68  37  31  2016         29.647059   
4030   Zaza Pachulia  DAL  2015/2016  76  40  36  2004         26.368421   

      DIST_FEET  DIST_MILES  ...  DIST_KM_DEF_PER_MIN  AVG_SPEED_MPH  \
0         71737          13  ...             0.053645           4.32   
1        741938         140  ...             0.052268           4.46   
2         87649          16  ...             0.052494           4.29   
3         12530           2  ...             0.055178           3.94   
4        205950          39  ...             0.056018           4.50   
...         ...         ...  ...                  ...            ...   
4026     158201          30  ...             0.052889           4.24   
4027      98364          18  ...             0.056063           4.57   
4028     880772         166  ...             0.055797           4.37   
4029     684888         129  ...             0.048935           3.87   
4030     707618         134  ...             0.051557           4.02   

      AVG_SPEED_MPH_OFF  AVG_SPEED_MPH_DEF  AVG_SPEED_KMH  AVG_SPEED_KMH_OFF  \
0                  4.46               4.17       6.952366           7.177674   
1                  5.05               3.88       7.177674           8.127187   
2                  4.52               4.04       6.904086           7.274235   
3                  4.14               3.77       6.340815           6.662684   
4                  4.61               4.39       7.242048           7.419076   
...                 ...                ...            ...                ...   
4026               4.45               4.02       6.823619           7.161581   
4027               4.93               4.21       7.354702           7.934066   
4028               4.62               4.13       7.032833           7.435169   
4029               3.93               3.80       6.228161           6.324722   
4030               4.09               3.95       6.469563           6.582217   

      AVG_SPEED_KMH_DEF  AVG_SPEED_KMH_PER_MIN  AVG_SPEED_KMH_OFF_PER_MIN  \
0              6.710964               0.036785                   0.037977   
1              6.244255               0.003760                   0.004257   
2              6.501750               0.029631                   0.031220   
3              6.067227               0.181166                   0.190362   
4              7.065020               0.013927                   0.014267   
...                 ...                    ...                        ...   
4026           6.469563               0.016018                   0.016811   
4027           6.775338               0.030142                   0.032517   
4028           6.646591               0.003067                   0.003243   
4029           6.115507               0.003089                   0.003137   
4030           6.356909               0.003228                   0.003285   

      AVG_SPEED_KMH_DEF_PER_MIN  
0                      0.035508  
1                      0.003271  
2                      0.027905  
3                      0.173349  
4                      0.013587  
...                         ...  
4026                   0.015187  
4027                   0.027768  
4028                   0.002899  
4029      

In [6]:
import pyspark.sql.functions as func

gd_df = df \
    .withColumn("AVG_MIN_PER_GAME", func.round(df["AVG_MIN_PER_GAME"], 2)) \
    .withColumn("AVG_SPEED_KMH", func.round(df["AVG_SPEED_KMH"], 2)) \
    .withColumn("AVG_SPEED_KMH_OFF", func.round(df["AVG_SPEED_KMH_OFF"], 2)) \
    .withColumn("AVG_SPEED_KMH_DEF", func.round(df["AVG_SPEED_KMH_DEF"], 2)) \
    .withColumn("AVG_SPEED_KMH_PER_MIN", func.round(df["AVG_SPEED_KMH_PER_MIN"], 2)) \
    .withColumn("AVG_SPEED_KMH_OFF_PER_MIN", func.round(df["AVG_SPEED_KMH_OFF_PER_MIN"], 2)) \
    .withColumn("AVG_SPEED_KMH_DEF_PER_MIN", func.round(df["AVG_SPEED_KMH_DEF_PER_MIN"], 2)) \
    .withColumn("DIST_KM", func.round(df["DIST_KM"], 2)) \
    .withColumn("DIST_KM_OFF", func.round(df["DIST_KM_OFF"], 2)) \
    .withColumn("DIST_KM_DEF", func.round(df["DIST_KM_DEF"], 2)) \
    .withColumn("DIST_KM_PER_MIN", func.round(df["DIST_KM_PER_MIN"], 2)) \
    .withColumn("DIST_KM_OFF_PER_MIN", func.round(df["DIST_KM_OFF_PER_MIN"], 2)) \
    .withColumn("DIST_KM_DEF_PER_MIN", func.round(df["DIST_KM_DEF_PER_MIN"], 2))
gd_df.show()

+----------------+----+---------+---+---+---+----+----------------+---------+----------+--------------+--------------+-------+-----------+-----------+---------------+-------------------+-------------------+-------------+-----------------+-----------------+-------------+-----------------+-----------------+---------------------+-------------------------+-------------------------+
|          PLAYER|TEAM|   SEASON| GP|  W|  L| MIN|AVG_MIN_PER_GAME|DIST_FEET|DIST_MILES|DIST_MILES_OFF|DIST_MILES_DEF|DIST_KM|DIST_KM_OFF|DIST_KM_DEF|DIST_KM_PER_MIN|DIST_KM_OFF_PER_MIN|DIST_KM_DEF_PER_MIN|AVG_SPEED_MPH|AVG_SPEED_MPH_OFF|AVG_SPEED_MPH_DEF|AVG_SPEED_KMH|AVG_SPEED_KMH_OFF|AVG_SPEED_KMH_DEF|AVG_SPEED_KMH_PER_MIN|AVG_SPEED_KMH_OFF_PER_MIN|AVG_SPEED_KMH_DEF_PER_MIN|
+----------------+----+---------+---+---+---+----+----------------+---------+----------+--------------+--------------+-------+-----------+-----------+---------------+-------------------+-------------------+-------------+-----------------+

In [10]:
#gd_df.show()
gd_df.toPandas()
gd_df.printSchema()

root
 |-- PLAYER: string (nullable = true)
 |-- TEAM: string (nullable = true)
 |-- SEASON: string (nullable = true)
 |-- GP: integer (nullable = true)
 |-- W: integer (nullable = true)
 |-- L: integer (nullable = true)
 |-- MIN: integer (nullable = true)
 |-- AVG_MIN_PER_GAME: double (nullable = true)
 |-- DIST_FEET: double (nullable = true)
 |-- DIST_MILES: double (nullable = true)
 |-- DIST_MILES_OFF: double (nullable = true)
 |-- DIST_MILES_DEF: double (nullable = true)
 |-- DIST_KM: double (nullable = true)
 |-- DIST_KM_OFF: double (nullable = true)
 |-- DIST_KM_DEF: double (nullable = true)
 |-- DIST_KM_PER_MIN: double (nullable = true)
 |-- DIST_KM_OFF_PER_MIN: double (nullable = true)
 |-- DIST_KM_DEF_PER_MIN: double (nullable = true)
 |-- AVG_SPEED_MPH: double (nullable = true)
 |-- AVG_SPEED_MPH_OFF: double (nullable = true)
 |-- AVG_SPEED_MPH_DEF: double (nullable = true)
 |-- AVG_SPEED_KMH: double (nullable = true)
 |-- AVG_SPEED_KMH_OFF: double (nullable = true)
 |-- AVG_S

In [9]:
gd_df = gd_df.withColumn("DIST_FEET", gd_df["DIST_FEET"].cast(DoubleType())) 
gd_df = gd_df.withColumn("DIST_MILES", gd_df["DIST_MILES"].cast(DoubleType())) 

In [11]:
# write to delta table
gd_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/fisico/gold/speed_distance/")

In [12]:
# check the results in the table
spark.table("gold.speed_distance").show()
spark.table("gold.speed_distance").count()

+----------------+----+---+---+---+----+----------------+---------+----------+--------------+--------------+-------+-----------+-----------+---------------+-------------------+-------------------+-------------+-----------------+-----------------+-------------+-----------------+-----------------+---------------------+-------------------------+-------------------------+---------+
|          PLAYER|TEAM| GP|  W|  L| MIN|AVG_MIN_PER_GAME|DIST_FEET|DIST_MILES|DIST_MILES_OFF|DIST_MILES_DEF|DIST_KM|DIST_KM_OFF|DIST_KM_DEF|DIST_KM_PER_MIN|DIST_KM_OFF_PER_MIN|DIST_KM_DEF_PER_MIN|AVG_SPEED_MPH|AVG_SPEED_MPH_OFF|AVG_SPEED_MPH_DEF|AVG_SPEED_KMH|AVG_SPEED_KMH_OFF|AVG_SPEED_KMH_DEF|AVG_SPEED_KMH_PER_MIN|AVG_SPEED_KMH_OFF_PER_MIN|AVG_SPEED_KMH_DEF_PER_MIN|   SEASON|
+----------------+----+---+---+---+----+----------------+---------+----------+--------------+--------------+-------+-----------+-----------+---------------+-------------------+-------------------+-------------+-----------------+----------

4031